In [28]:
import pandas as pd
import numpy as np 
import glob
import cv2
import tensorflow as tf
from sklearn import preprocessing

In [29]:
sample_breed = pd.read_csv("sample.csv") 
count = 0
img = []    
for index, row in sample_breed.iterrows():
    file = row['id']
    breed = row['breed']
#     print(file)
    img.append(cv2.imread('processed/' + file ,0))
#     a = cv2.imread('processed/' + file ,0) 
#     cv2.imshow('image',a)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    


In [49]:
# enc = preprocessing.LabelEncoder()

# # 2. FIT
# enc.fit(sample_breed["breed"])

# 3. Transform
# onehotlabels = enc.transform(sample_breed["breed"])
# onehotlabels
# sample_breed["breed"] = onehotlabels
# sample_breed["breed"]
s = pd.Series(sample_breed['breed'])
s = pd.get_dummies(s)


In [50]:
sample_breed["img"] = img

train_x = sample_breed["img"].iloc[:200]
train_y = s.iloc[:200]
test_x = sample_breed["img"].iloc[200:]
test_y = s.iloc[:200]


In [51]:
np.shape(sample_breed["img"][0])

(224, 224)

In [52]:
learning_rate = 0.001
num_steps = 200
batch_size = 128
display_step = 10


In [53]:
num_input = 64 * 64  
num_classes = 2
dropout = 0.75 

In [54]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) 

In [55]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation 
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


In [56]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


In [57]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 64, 64, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [58]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([16*16*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [59]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


In [60]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [61]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


In [62]:
# train_x = sample_breed["img"].iloc[:200]
# train_y = sample_breed["breed"].iloc[:200]
# test_x = sample_breed["img"].iloc[200:]
# test_y = sample_breed["breed"].iloc[:200]


In [64]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
#         batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: train_x, Y: train_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_x,
                                                                 Y: train_y.values,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    
    # Calculate accuracy for 256 MNIST test images
#     print("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
#                                       Y: mnist.test.labels[:256],
#                                       keep_prob: 1.0}))

ValueError: setting an array element with a sequence.